Este notebook es para generar el paso de tener parrafos o un texto segmentado y generar codes para estos o un hasthag, despues de generar el hasthag vamos a vectorizarlos en altas dimensiones y aplicaremos HBDSCAN ahi mismo en las altas dimensiones, despues ya con los clusters vamos a pedir al LLM un theme para el cluster, para darles nombre y al final los visualizamos


In [4]:
pip install sentence-transformers umap-learn scikit-learn matplotlib seaborn pandas numpy optuna ollama nbformat>=4.2 

Note: you may need to restart the kernel to use updated packages.


**Exploratory analysis**

In [5]:
import pandas as pd

transcripts_Armando = "https://raw.githubusercontent.com/3eltran23/ATA-LMM_study/refs/heads/main/data/transcripts.csv"
transcripts_Armando = pd.read_csv(transcripts_Armando)

codes_Armando = "https://raw.githubusercontent.com/3eltran23/ATA-LMM_study/refs/heads/main/data/combined_codes.csv"
codes_Armando = pd.read_csv(codes_Armando)

Hockey_Interviews = "https://raw.githubusercontent.com/HiramZ04/Cal-State-UMAP_Clustering/refs/heads/main/data/National-Hockey-League-Interviews.csv"
Hockey_Interviews = pd.read_csv(Hockey_Interviews)

callrecordings = "https://raw.githubusercontent.com/HiramZ04/Cal-State-UMAP_Clustering/refs/heads/main/data/call_recordings.csv"
callrecordings = pd.read_csv(callrecordings)

sentiment = "https://raw.githubusercontent.com/HiramZ04/Cal-State-UMAP_Clustering/refs/heads/main/data/sentiment_analysis.csv"
sentiment = pd.read_csv(sentiment)

transcripts_Armando.head(5)

,Filename,Index,Segment/Excerpt
0,p008-parent,1,Parent:\tI think you had asked me what my conc...
1,p008-parent,2,for Reed Richards having it.
2,p008-parent,3,"So, him losing it, it kept making damage somehow."
3,p008-parent,4,Those were really my biggest concerns.\nInterv...
4,p008-parent,5,Do you or your son?\nParent:\tThe three of us ...


In [6]:
codes_Armando.head(5)

,Codes,Source
0,Reinforcing school dynamic at home,p008-parent
1,Functionalities of smartwatch,p008-parent
2,Daily or weekly goals,p008-parent
3,Sharing information with separated parents,p008-parent
4,Pick up of watch and phone,p008-parent


In [7]:
Hockey_Interviews.head(5)

,RowId,team1,team2,date,name,job,text
0,0,blackhawks,lightning,2015-06-02,stan bowman,other,well we're very fortunate to have the players ...
1,1,blackhawks,lightning,2015-06-02,steve yzerman,other,no we didn't really set a timeline on it i wou...
2,2,blackhawks,lightning,2015-06-03,antoine vermette,player,that's a good question i don't recall specific...
3,3,blackhawks,lightning,2015-06-03,joel quenneville,coach,yeah we got better as the game went on i thoug...
4,4,blackhawks,lightning,2015-06-03,jon cooper,coach,i don't know i think the way i'd look at the g...


In [8]:
callrecordings.head(5)

,id,Type,Sentiment,Name,Order Number,Product Number,Transcript
0,call_recording_01,Product Inquiry,Neutral,Sarah Miller,NaN,AC-7892,"Hello, I'm Sarah Miller. I'm calling to inquir..."
1,call_recording_02,Complaint,Angry,John Davis,123456.0,FR-4401,I am extremely dissatisfied with my recent ord...
2,call_recording_03,Technical Issue,Frustrated,Maria Rodriguez,987654.0,LAP-2110,"Hi, this is Maria Rodriguez. I'm having troubl..."
3,call_recording_04,Compliment,Happy,Robert Smith,246801.0,DW-6543,I just wanted to call and say how pleased I am...
4,call_recording_05,Order Placement,Neutral,Jessica Brown,NaN,OV-1357 & MW-8642,"Hello, my name is Jessica Brown. I'd like to p..."


In [9]:
sentiment.head(5)

,Year,Month,Day,Time of Tweet,text,sentiment,Platform
0,2018,8,18,morning,What a great day!!! Looks like dream.,positive,Twitter
1,2018,8,18,noon,"I feel sorry, I miss you here in the sea beach",positive,Facebook
2,2017,8,18,night,Don't angry me,negative,Facebook
3,2022,6,8,morning,We attend in the class just for listening teac...,negative,Facebook
4,2022,6,8,noon,"Those who want to go, let them go",negative,Instagram


**Preparando el system para el LLM**

In [10]:
from ollama import chat
# We start passing a transcript to LLM
# We use gemma2:9b  

prompt_template = """
    You are a qualitative research analyst. Read one interview segment and produce EXACTLY ONE thematic label.

Requirements for the label:
- 2–6 words, concise and descriptive
- Neutral, observable phrasing (no opinions or causes)
- Title Case, no punctuation, no quotes
- Include at least one salient noun from the segment (e.g., product name, parameter, entity)


    Output Format:
    - ONLY 1 CODE and nothing else

    """

In [11]:
# le pasamos las palabras al llm 
# CALL CENTER DATA SET

sentences_calls = callrecordings["Transcript"].astype(str).tolist()
    
labels_calls = []
GEN_OPTS = {"temperature": 0.0, "top_p": 1.0, "repeat_penalty": 1.2}

for i in range(len(sentences_calls)): 
# Pasamos el prompt con un system para que nos regrese el output que deseamos
    messages = [
        {
        'role':'system','content':prompt_template
        },
        {
        'role':'user', 'content':f'This is the transcript i need the codes for: {sentences_calls[i]}',
        },   
    ]

    label = chat(model = "gemma2:9b", messages=messages, options=GEN_OPTS)
    labels_calls.append(label['message']['content'])
    print("LLM response: ",label['message']['content'])

print("Lista completa de codes: ",labels_calls)


LLM response:  Product Feature Inquiry 

LLM response:  Damaged Refrigerator Complaint 

LLM response:  Laptop Connectivity Issues 

LLM response:  Customer Satisfaction DW-6543  

LLM response:  Customer Order Inquiry 

LLM response:  Wireless Speaker Functionality 

LLM response:  Order Delivery Delay 

LLM response:  Router Network Setup 

LLM response:  Customer Order Inquiry 

LLM response:  Customer Satisfaction With PR9876  

LLM response:  Technical Data Request 

LLM response:  Color Discrepancy Concern  

LLM response:  System Outage Impact 

LLM response:  Product Inquiry Weekend Delivery 

LLM response:  Customer Satisfaction With CAMERA-PRO-1 

LLM response:  Tablet Functionality Inquiry 

LLM response:  WasherDryer Combo Malfunction 

LLM response:  Software Suite Pro Installation Issues 

LLM response:  Office Furniture Procurement 

LLM response:  Customer Satisfaction With Television  

Lista completa de codes:  ['Product Feature Inquiry \n', 'Damaged Refrigerator Comp

**construimos thematic labels con LLMs sobre los textos**

In [12]:
# le pasamos las palabras al llm 
# CALL CENTER DATA SET

sentences_sentiment = sentiment["text"].astype(str).tolist()
    
labels_sentiment = []
GEN_OPTS = {"temperature": 0.0, "top_p": 1.0, "repeat_penalty": 1.2}

for i in range(len(sentences_sentiment)): 
# Pasamos el prompt con un system para que nos regrese el output que deseamos
    messages = [
        {
        'role':'system','content':prompt_template
        },
        {
        'role':'user', 'content':f'This is the transcript i need the codes for: {sentences_sentiment[i]}',
        },   
    ]

    label = chat(model = "gemma2:9b", messages=messages, options=GEN_OPTS)
    labels_sentiment.append(label['message']['content'])
    print("LLM response: ",label['message']['content'])

print("Lista completa de codes: ",labels_sentiment)


LLM response:  Positive User Experience  

LLM response:  Emotional Connection To Place  

LLM response:  Emotional Response To Situation  

LLM response:  Classroom Learning Experience   

LLM response:  Individual Choice Expression  

LLM response:  Sleep And Mood  

LLM response:  Positive Baby Interactions  

LLM response:  Emotional Response To Product  

LLM response:  Participant Interaction Dynamics   

LLM response:  Boat Travel Experience  

LLM response:  Conceptual Metaphor Usage  

LLM response:  Musical Preference Expression  

LLM response:  Ink Supply Depletion  

LLM response:  Vocal Impairment Impact  

LLM response:  Emotional Absence  

LLM response:  User Request For Privacy  

LLM response:  User Disinterest  

LLM response:  Recent Illness Experience  

LLM response:  Positive Past Employment  

LLM response:  Romantic Feelings Expression  

LLM response:  Concern For Missing Person 

LLM response:  Play Experiences With Children  

LLM response:  Household Prepa

In [13]:
print("Cantidad de thematic labels generados: ",len(labels_sentiment))

Cantidad de thematic labels generados:  499


In [ ]:
# Importamos a un csv solo los labels

import pandas as pd

pd.DataFrame({"label": labels_sentiment}).to_csv(
    "labels_only.csv", index=False, encoding="utf-8-sig"
)


In [ ]:
# Importamos a un csv indice, sentiment analysis y su label generado por LLM

pd.DataFrame({
    "idx": range(len(labels_sentiment)),
    "text": sentences_sentiment[:len(labels_sentiment)],
    "label": labels_sentiment
}).to_csv("labels_sentiment.csv", index=False, encoding="utf-8-sig")

**Vectorizamos los labels**

In [14]:
# Despues de generar los thematic labels vectorizamos y hacemos cluster con HBDSCAN
from sentence_transformers import SentenceTransformer

ST_model = SentenceTransformer('all-MiniLM-L6-v2')
sentiment_embedded = ST_model.encode(labels_sentiment, convert_to_numpy=True)

In [15]:
print(sentiment_embedded)
len(sentiment_embedded)

[[-0.03031873  0.05057195 -0.01491859 ... -0.02037648 -0.00163911
   0.0413107 ]
 [ 0.07348786 -0.02340687  0.03307682 ...  0.1238998  -0.05009362
  -0.03497658]
 [-0.00699909  0.05914862  0.00685273 ...  0.06271049 -0.07404987
  -0.03521739]
 ...
 [-0.01479522 -0.02378483 -0.0498682  ...  0.02121904 -0.07809275
   0.04581887]
 [-0.02023393 -0.02837187  0.0510417  ... -0.02707418  0.0074882
   0.03977657]
 [ 0.06275199  0.03894886  0.0391977  ...  0.05683921 -0.08846852
   0.05103279]]


499

**Graficamos el embbedded space**

In [25]:
# Requisitos: sentiment_embedded (np.ndarray n×d) y labels_sentiment (list[str] de largo n)

import numpy as np
import pandas as pd
import plotly.express as px
from umap import UMAP

# Seguridad: que coincidan tamaños
assert len(labels_sentiment) == sentiment_embedded.shape[0], "Dim mismatch"

# Reducir a 2D solo para visualizar
X2 = UMAP(n_components=2, metric="cosine", random_state=42).fit_transform(sentiment_embedded)

# DataFrame para el plot (incluye 'code' = tus labels)
df_plot = pd.DataFrame({
    "x1": X2[:, 0],
    "x2": X2[:, 1],
    "code": [str(c) if c is not None else "" for c in labels_sentiment],
})

fig = px.scatter(
    df_plot, x="x1", y="x2",
    hover_data=["code"]   # <-- aquí se ve el label en el hover
)
fig.update_layout(xaxis_title="x1", yaxis_title="x2")
fig.show()


c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



**Implementamos HDBSCAN**

In [ ]:
from hdbscan import HDBSCAN
from sklearn.preprocessing import normalize


hdb = HDBSCAN(min_cluster_size=16, min_samples=1 ).fit(sentiment_embedded)
labels = hdb.labels_


print(labels)



[ 1 -1  1 -1 -1 -1 -1  1  1  2 -1  2 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1
 -1 -1 -1 -1  2 -1 -1  1 -1 -1 -1  2 -1 -1 -1 -1 -1  1 -1  0  0 -1  2 -1
  1 -1 -1 -1 -1  0 -1  1  1 -1  0  0 -1 -1 -1  1 -1 -1  1 -1  1 -1 -1 -1
 -1  1  1  2 -1  2 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1
 -1  1 -1 -1 -1  2 -1 -1 -1 -1 -1  1 -1  0  0 -1  2  0  0  1 -1 -1  1 -1
 -1 -1  1 -1  1  1 -1 -1 -1  2 -1 -1  2 -1 -1  1 -1  2  2  0 -1 -1  0  2
 -1  1  2 -1  1 -1  1 -1  1 -1  2  1 -1 -1 -1 -1  1 -1  2 -1 -1 -1  1 -1
  1  2  1  0 -1  1 -1  1 -1  1 -1  1 -1  1  1 -1 -1  2 -1  1 -1  1  1 -1
 -1  2 -1 -1 -1 -1 -1 -1  1  0 -1  0  2  0  0  0  0  1 -1 -1  2 -1 -1 -1
  1 -1 -1 -1 -1 -1 -1  2  1 -1  1 -1  2  0 -1  2  2  0 -1 -1  0  2  1  1
 -1 -1  1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1  1 -1
  1  1 -1  1 -1  1 -1  1 -1  1 -1  1  1 -1 -1  2 -1  1 -1  1  1 -1 -1  2
 -1 -1 -1 -1 -1 -1  1  0 -1  0  2 -1  1  1  1 -1 -1  2  1  1  2  1  2 -1
  0 -1  0 -1 -1  1 -1  0 -1  1  2 -1  2  0  1 -1  1

c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



**Graficamos clusters**

In [21]:
import pandas as pd
import plotly.express as px

df_plot = pd.DataFrame({
    "x1": X2[:, 0],           # reusa la proyeccion 2D de la celda anterior
    "x2": X2[:, 1],
    "cluster": labels,    # −1 = ruido
    "code": labels_sentiment
})

# opcion: mostrar "noise" en la leyenda
df_plot["cluster_str"] = df_plot["cluster"].astype(str).replace({"-1": "noise"})

fig = px.scatter(
    df_plot, x="x1", y="x2",
    color="cluster_str",               # o usa "cluster" si prefieres numerico
    hover_data=["code"],
    color_discrete_sequence=px.colors.qualitative.Plotly
)
fig.update_layout(
    xaxis_title="x1", yaxis_title="x2",
    legend_title_text="cluster",
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, xanchor="center", x=0.5)
)
fig.show()


**Implementamos optuna para el HDBSCAN** 

In [ ]:
#Vimos mucho ruido entonces ahora usamos el Optuna para ver si prevalece el ruido con los mejores parametros 

**Pedimos el thematic analysis para clasificar y nombrar los clusters al LLM**

In [26]:
# === Theme naming por cluster con Ollama ===
# Requiere: labels_sentiment (list[str]) y labels_hdb (np.ndarray o list[int])
# Salida: df_cluster_themes (tabla con nombres) y cluster_name_por_fila (list[str])

import json, random
import pandas as pd
from collections import defaultdict
from ollama import chat

MODEL = "gemma2:9b"   # o "gemma2:9b"
GEN_OPTS = {
    "temperature": 0.0,
    "top_p": 1.0,
    "repeat_penalty": 1.2,
    "num_predict": 96,   # suficiente para nombre + definición
}

SYSTEM_PROMPT = """
You are a qualitative research analyst.
Given several short codes that belong to ONE cluster, propose:
- a concise cluster name (2–5 words, Title Case, no punctuation),
- a one-sentence definition describing the common theme.

Return STRICT JSON in ONE line:
{"cluster_name":"<2-5 words>", "definition":"<one sentence>"}
Do not include any extra text.
"""

def sample_codes(codes, k=12):
    """muestra sin repetidos (hasta k)"""
    uniq = list(dict.fromkeys([c.strip() for c in codes if str(c).strip()]))
    if len(uniq) <= k:
        return uniq
    return random.sample(uniq, k)

def ask_theme_ollama(codes_sample):
    # pasa los codes como bullets para que el LLM vea variedad
    bullets = "\n".join(f"- {c}" for c in codes_sample)
    user = f"Here are representative codes from one cluster:\n{bullets}\n\nReturn JSON only."
    resp = chat(
        model=MODEL,
        messages=[{"role":"system","content":SYSTEM_PROMPT},
                  {"role":"user","content":user}],
        options=GEN_OPTS,
        keep_alive=-1,
    )
    txt = resp["message"]["content"].strip()
    # intenta parsear JSON
    try:
        data = json.loads(txt)
        name = str(data.get("cluster_name","")).strip()
        definition = str(data.get("definition","")).strip()
        if not name:
            name = "Unnamed Theme"
        if not definition:
            definition = "No definition provided."
        return name, definition, txt
    except json.JSONDecodeError:
        # fallback: usa el texto completo como nombre
        return txt[:60], "LLM returned non-JSON; truncated as name.", txt

# --- agrupa codes por cluster ---
codes_por_cluster = defaultdict(list)
for lab, clu in zip(labels_sentiment, labels):
    if clu == -1:
        continue  # omite ruido
    codes_por_cluster[int(clu)].append(lab)

# --- pide nombre/definición por cluster ---
rows = []
for clu_id, codes_list in sorted(codes_por_cluster.items()):
    sample = sample_codes(codes_list, k=12)
    name, definition, raw = ask_theme_ollama(sample)
    rows.append({
        "cluster_id": clu_id,
        "n_items": len(codes_list),
        "sampled_codes": sample,
        "cluster_name": name,
        "definition": definition,
        "raw_llm": raw,      # por si quieres inspeccionar lo que devolvió
    })

df_cluster_themes = pd.DataFrame(rows).sort_values("cluster_id").reset_index(drop=True)
print(df_cluster_themes[["cluster_id","n_items","cluster_name"]])

# --- mapea nombre a cada fila (alineado a labels_hdb) ---
name_map = {row.cluster_id: row.cluster_name for _, row in df_cluster_themes.iterrows()}
cluster_name_por_fila = [name_map.get(int(c), "noise") if c != -1 else "noise" for c in labels]

# Ejemplo: si tienes un DataFrame con tus puntos, añade la columna
# df_plot["cluster_name"] = cluster_name_por_fila

# Guarda opcional
# df_cluster_themes.to_csv("cluster_themes.csv", index=False, encoding="utf-8-sig")


   cluster_id  n_items                   cluster_name
0           0       39         Daily Life And Leisure
1           1      122  Social Connection And Emotion
2           2       56      Social Identity Formation


**Graficamos los clusters con nombre**

In [28]:
# pip install umap-learn plotly
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from umap import UMAP

# 1) Proyección a 2D (solo para visualizar)
X2 = UMAP(n_components=2, metric="cosine", random_state=42).fit_transform(sentiment_embedded)

# 2) Construye la lista de nombres de tema por punto
if 'cluster_name_por_fila' in globals():
    themes = cluster_name_por_fila  # ya la tienes de la celda anterior
else:
    # map desde df_cluster_themes
    name_map = {int(r.cluster_id): r.cluster_name for _, r in df_cluster_themes.iterrows()}
    themes = [name_map.get(int(c), "noise") if c != -1 else "noise" for c in labels_hdb]

# 3) DataFrame para plotear
df_plot = pd.DataFrame({
    "x1": X2[:, 0],
    "x2": X2[:, 1],
    "theme": themes,                 # nombre del cluster
    "code": labels_sentiment,        # tu label/código por punto
})

# 4) Scatter coloreado por tema con hover (code + theme)
fig = px.scatter(
    df_plot, x="x1", y="x2",
    color="theme",
    hover_data=["code", "theme"],
    color_discrete_sequence=px.colors.qualitative.Plotly
)

# (Opcional) Si quieres que el "ruido" se llame noise siempre:
# df_plot["theme"] = df_plot["theme"].astype(str)

fig.update_layout(
    xaxis_title="x1", yaxis_title="x2",
    legend_title_text="Themes",
    legend=dict(orientation="h", yanchor="bottom", y=-0.25, xanchor="center", x=0.5)
)



fig.show()


c:\Users\Hiram\AppData\Local\Programs\Python\Python312\Lib\site-packages\umap\umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

